<a href="https://colab.research.google.com/github/Kyamzzz/GISMA/blob/main/DZ_RFM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the two parts of the offline data
offline_data_part1 = pd.read_excel("Offline_Raw.xlsx")
offline_data_part2 = pd.read_excel("Offline_Raw_1.xlsx")

# Combine the two parts into a single dataframe
offline_data_combined = pd.concat([offline_data_part1, offline_data_part2], ignore_index=True)

# Display the first few rows to confirm loading and merging
offline_data_combined.head()


,PersonGUID,email,orderTime,ArtikelID,ArtikelNameD,Umsatz,Menge
0,4CB1C004-B1A2-498E-B0D1-536D63C421BC,e.duebi@gmx.ch,2022-05-10,107179900000,DELIZIO LEGGERO 48KAP,16.8,1
1,4CB1C004-B1A2-498E-B0D1-536D63C421BC,e.duebi@gmx.ch,2022-05-10,107182100000,DELIZIO CREMA 48KAP,33.6,2
2,4CB1C004-B1A2-498E-B0D1-536D63C421BC,e.duebi@gmx.ch,2022-05-10,107184700000,DELIZIO DECAFFEINAT.48KAP,16.8,1
3,4CB1C004-B1A2-498E-B0D1-536D63C421BC,e.duebi@gmx.ch,2022-09-23,107179900000,DELIZIO LEGGERO 48KAP,19.8,1
4,4CB1C004-B1A2-498E-B0D1-536D63C421BC,e.duebi@gmx.ch,2022-09-23,107182100000,DELIZIO CREMA 48KAP,39.6,2


In [ ]:
online_data_path = "Online_Raw.xlsx"
online_data = pd.read_excel(online_data_path, parse_dates=['orderTime'])


In [ ]:
# Extract unique emails from both datasets
unique_emails_online = set(online_data['email'])
unique_emails_offline = set(offline_data_combined['email'])

# Find the common emails between online and offline datasets
common_emails = unique_emails_online.intersection(unique_emails_offline)

# Filter the datasets to only include rows with the common emails
filtered_online_data = online_data[online_data['email'].isin(common_emails)]
filtered_offline_data = offline_data_combined[offline_data_combined['email'].isin(common_emails)]

# Display the number of unique emails in the filtered datasets
num_unique_emails = len(common_emails)
num_unique_emails


28397

In [ ]:
# Group by email and get the latest orderTime for both the Online and Offline data
latest_online_order = filtered_online_data.groupby('email')['orderTime'].max().reset_index()
latest_offline_order = filtered_offline_data.groupby('email')['orderTime'].max().reset_index()

# Extract the GUIDs for the emails from the Offline data
guids = filtered_offline_data.drop_duplicates(subset='email')[['email', 'PersonGUID']]

# Merge the datasets
consolidated_data = pd.merge(latest_online_order, latest_offline_order, on='email', how='inner')
consolidated_data = pd.merge(consolidated_data, guids, on='email', how='inner')

# Rename the columns for clarity
consolidated_data = consolidated_data.rename(columns={
    'orderTime_x': 'orderTime_Online',
    'orderTime_y': 'orderTime_Offline',
    'PersonGUID': 'GUID'
})

# Select the desired columns for final output
consolidated_data = consolidated_data[['email', 'GUID', 'orderTime_Online', 'orderTime_Offline']]
consolidated_data

,email,GUID,orderTime_Online,orderTime_Offline
0,07zaf59@gmail.com,4325D934-6F38-4999-A87C-57605DC80B56,2023-03-01 10:19:29,2023-06-03
1,09webmail@gmail.com,B2206CC0-A8FC-4FFF-8FCB-D907DEC484D6,2023-06-16 05:52:48,2023-04-05
2,0roland.eberle0@gmail.com,7256FB03-2837-4D8D-9A9C-E2D65715CEC9,2022-11-15 09:42:31,2022-03-12
3,100571@gmx.ch,B0B3E49A-49B8-48A2-ACB2-1B7DBACA1697,2022-11-27 08:25:34,2022-11-04
4,1016@gmx.ch,64B02A11-8921-4546-84E5-8CDC917BD251,2022-07-03 14:43:14,2022-05-27
...,...,...,...,...
28392,zwiker@gmx.ch,2F45C76A-B3D9-4DB3-8A29-B8CE8CC21B52,2023-04-12 07:39:34,2023-04-08
28393,zwimpfer.ruth@bluewin.ch,509E82ED-5366-44A2-B3AC-23ABE884DB28,2023-05-20 15:41:43,2023-03-25
28394,zwyer-zettel@bluewin.ch,DAC6274F-A7DD-4BC7-BE79-7AC9AC05BC05,2022-05-05 17:42:37,2023-04-08
28395,zy.ame@hotmail.ch,EBFE76CE-E94C-423C-B257-CB7BE3D18472,2022-06-09 13:43:11,2022-11-15


In [ ]:
consolidated_data['orderTime_Online'] = consolidated_data['orderTime_Online'].dt.strftime('%d.%m.%Y')
consolidated_data['orderTime_Offline'] = consolidated_data['orderTime_Offline'].dt.strftime('%d.%m.%Y')
consolidated_data

,email,GUID,orderTime_Online,orderTime_Offline
0,07zaf59@gmail.com,4325D934-6F38-4999-A87C-57605DC80B56,01.03.2023,03.06.2023
1,09webmail@gmail.com,B2206CC0-A8FC-4FFF-8FCB-D907DEC484D6,16.06.2023,05.04.2023
2,0roland.eberle0@gmail.com,7256FB03-2837-4D8D-9A9C-E2D65715CEC9,15.11.2022,12.03.2022
3,100571@gmx.ch,B0B3E49A-49B8-48A2-ACB2-1B7DBACA1697,27.11.2022,04.11.2022
4,1016@gmx.ch,64B02A11-8921-4546-84E5-8CDC917BD251,03.07.2022,27.05.2022
...,...,...,...,...
28392,zwiker@gmx.ch,2F45C76A-B3D9-4DB3-8A29-B8CE8CC21B52,12.04.2023,08.04.2023
28393,zwimpfer.ruth@bluewin.ch,509E82ED-5366-44A2-B3AC-23ABE884DB28,20.05.2023,25.03.2023
28394,zwyer-zettel@bluewin.ch,DAC6274F-A7DD-4BC7-BE79-7AC9AC05BC05,05.05.2022,08.04.2023
28395,zy.ame@hotmail.ch,EBFE76CE-E94C-423C-B257-CB7BE3D18472,09.06.2022,15.11.2022


In [ ]:
import numpy as np

consolidated_data['orderTime_Online'] = pd.to_datetime(consolidated_data['orderTime_Online'], dayfirst=True)
consolidated_data['orderTime_Offline'] = pd.to_datetime(consolidated_data['orderTime_Offline'], dayfirst=True)

end_date = pd.Timestamp('2023-06-30')

# Define the function to categorize recency into buckets
def categorize_recency(months):
    if months <= 3:
        return '0-3'
    elif months <= 6:
        return '3-6'
    elif months <= 9:
        return '6-9'
    elif months <= 12:
        return '9-12'
    elif months <= 15:
        return '12-15'
    else:
        return '15-18'

# Calculate the recency in months for both online and offline data
consolidated_data['Months_Online'] = ((end_date - consolidated_data['orderTime_Online']) / np.timedelta64(1, 'M')).astype(int).apply(categorize_recency)
consolidated_data['Months_Offline'] = ((end_date - consolidated_data['orderTime_Offline']) / np.timedelta64(1, 'M')).astype(int).apply(categorize_recency)

# For Recency in days
consolidated_data['Recency_Online'] = (end_date - consolidated_data['orderTime_Online']).dt.days
consolidated_data['Recency_Offline'] = (end_date - consolidated_data['orderTime_Offline']).dt.days
consolidated_data


,email,GUID,orderTime_Online,orderTime_Offline,Months_Online,Months_Offline,Recency_Online,Recency_Offline
0,07zaf59@gmail.com,4325D934-6F38-4999-A87C-57605DC80B56,2023-03-01,2023-06-03,0-3,0-3,121,27
1,09webmail@gmail.com,B2206CC0-A8FC-4FFF-8FCB-D907DEC484D6,2023-06-16,2023-04-05,0-3,0-3,14,86
2,0roland.eberle0@gmail.com,7256FB03-2837-4D8D-9A9C-E2D65715CEC9,2022-11-15,2022-03-12,6-9,12-15,227,475
3,100571@gmx.ch,B0B3E49A-49B8-48A2-ACB2-1B7DBACA1697,2022-11-27,2022-11-04,6-9,6-9,215,238
4,1016@gmx.ch,64B02A11-8921-4546-84E5-8CDC917BD251,2022-07-03,2022-05-27,9-12,12-15,362,399
...,...,...,...,...,...,...,...,...
28392,zwiker@gmx.ch,2F45C76A-B3D9-4DB3-8A29-B8CE8CC21B52,2023-04-12,2023-04-08,0-3,0-3,79,83
28393,zwimpfer.ruth@bluewin.ch,509E82ED-5366-44A2-B3AC-23ABE884DB28,2023-05-20,2023-03-25,0-3,0-3,41,97
28394,zwyer-zettel@bluewin.ch,DAC6274F-A7DD-4BC7-BE79-7AC9AC05BC05,2022-05-05,2023-04-08,12-15,0-3,421,83
28395,zy.ame@hotmail.ch,EBFE76CE-E94C-423C-B257-CB7BE3D18472,2022-06-09,2022-11-15,9-12,6-9,386,227


In [ ]:
# Calculate Recency for Online and Offline data
consolidated_data['Recency_Online'] = (end_date - consolidated_data['orderTime_Online']).dt.days
consolidated_data['Recency_Offline'] = (end_date - consolidated_data['orderTime_Offline']).dt.days

# Display the dataframe to confirm changes
consolidated_data[['email', 'orderTime_Online', 'Recency_Online', 'orderTime_Offline', 'Recency_Offline']].head()


,email,orderTime_Online,Recency_Online,orderTime_Offline,Recency_Offline
0,07zaf59@gmail.com,2023-03-01,121,2023-06-03,27
1,09webmail@gmail.com,2023-06-16,14,2023-04-05,86
2,0roland.eberle0@gmail.com,2022-11-15,227,2022-03-12,475
3,100571@gmx.ch,2022-11-27,215,2022-11-04,238
4,1016@gmx.ch,2022-07-03,362,2022-05-27,399


In [ ]:
# Calculate Frequency for Online data
freq_online = online_data.groupby('email')['orderTime'].nunique().reset_index()
freq_online.columns = ['email', 'Frequency_Online']

# Calculate Frequency for Offline data using 'offline_data_combined'
freq_offline = offline_data_combined.groupby('email')['orderTime'].nunique().reset_index()
freq_offline.columns = ['email', 'Frequency_Offline']

# Merge the frequencies with our consolidated_data dataframe
consolidated_data = consolidated_data.merge(freq_online, on='email', how='left')
consolidated_data = consolidated_data.merge(freq_offline, on='email', how='left')

consolidated_data.head()



,email,GUID,orderTime_Online,orderTime_Offline,Months_Online,Months_Offline,Recency_Online,Recency_Offline,Frequency_Online,Frequency_Offline
0,07zaf59@gmail.com,4325D934-6F38-4999-A87C-57605DC80B56,2023-03-01,2023-06-03,0-3,0-3,121,27,2,15
1,09webmail@gmail.com,B2206CC0-A8FC-4FFF-8FCB-D907DEC484D6,2023-06-16,2023-04-05,0-3,0-3,14,86,3,13
2,0roland.eberle0@gmail.com,7256FB03-2837-4D8D-9A9C-E2D65715CEC9,2022-11-15,2022-03-12,6-9,12-15,227,475,2,1
3,100571@gmx.ch,B0B3E49A-49B8-48A2-ACB2-1B7DBACA1697,2022-11-27,2022-11-04,6-9,6-9,215,238,2,8
4,1016@gmx.ch,64B02A11-8921-4546-84E5-8CDC917BD251,2022-07-03,2022-05-27,9-12,12-15,362,399,1,3


In [ ]:
# Calculate Monetary for Online data
monetary_online = online_data.groupby('email')['Amount'].sum().reset_index()
monetary_online.columns = ['email', 'Monetary_Online']

# Calculate Monetary for Offline data
monetary_offline = offline_data_combined.groupby('email')['Umsatz'].sum().reset_index()
monetary_offline.columns = ['email', 'Monetary_Offline']

# Merge the monetary values into the consolidated data
consolidated_data = pd.merge(consolidated_data, monetary_online, on='email', how='left')
consolidated_data = pd.merge(consolidated_data, monetary_offline, on='email', how='left')

# Display the first few rows to confirm
consolidated_data.head()


,email,GUID,orderTime_Online,orderTime_Offline,Months_Online,Months_Offline,Recency_Online,Recency_Offline,Frequency_Online,Frequency_Offline,Monetary_Online,Monetary_Offline
0,07zaf59@gmail.com,4325D934-6F38-4999-A87C-57605DC80B56,2023-03-01,2023-06-03,0-3,0-3,121,27,2,15,31.1,351.96
1,09webmail@gmail.com,B2206CC0-A8FC-4FFF-8FCB-D907DEC484D6,2023-06-16,2023-04-05,0-3,0-3,14,86,3,13,176.5,329.96
2,0roland.eberle0@gmail.com,7256FB03-2837-4D8D-9A9C-E2D65715CEC9,2022-11-15,2022-03-12,6-9,12-15,227,475,2,1,128.3,19.80
3,100571@gmx.ch,B0B3E49A-49B8-48A2-ACB2-1B7DBACA1697,2022-11-27,2022-11-04,6-9,6-9,215,238,2,8,138.2,180.12
4,1016@gmx.ch,64B02A11-8921-4546-84E5-8CDC917BD251,2022-07-03,2022-05-27,9-12,12-15,362,399,1,3,170.5,110.94


In [ ]:
def uniform_recency_rating(days):
    if days <= 109:
        return 5
    elif days <= 218:
        return 4
    elif days <= 327:
        return 3
    elif days <= 436:
        return 2
    else:
        return 1

consolidated_data['Rating R_Online'] = consolidated_data['Recency_Online'].apply(uniform_recency_rating)
consolidated_data['Rating R_Offline'] = consolidated_data['Recency_Offline'].apply(uniform_recency_rating)

# To view the first few rows of the dataframe
print(consolidated_data[['email', 'Recency_Online', 'Rating R_Online', 'Recency_Offline', 'Rating R_Offline']].head())



                       email  Recency_Online  Rating R_Online  \
0          07zaf59@gmail.com             121                4   
1        09webmail@gmail.com              14                5   
2  0roland.eberle0@gmail.com             227                3   
3              100571@gmx.ch             215                4   
4                1016@gmx.ch             362                2   

   Recency_Offline  Rating R_Offline  
0               27                 5  
1               86                 5  
2              475                 1  
3              238                 3  
4              399                 2  


In [ ]:
# Analyzing the distribution of Frequency for Online data
online_freq_desc = consolidated_data['Frequency_Online'].describe()

# Analyzing the distribution of Frequency for Offline data
offline_freq_desc = consolidated_data['Frequency_Offline'].describe()

print("Online Frequency Distribution:\n", online_freq_desc, "\n")
print("Offline Frequency Distribution:\n", offline_freq_desc)


Online Frequency Distribution:
 count    28397.000000
mean         2.669965
std          2.468800
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         36.000000
Name: Frequency_Online, dtype: float64 

Offline Frequency Distribution:
 count    28397.000000
mean        14.527626
std         14.829193
min          1.000000
25%          4.000000
50%         10.000000
75%         20.000000
max        236.000000
Name: Frequency_Offline, dtype: float64


In [ ]:
# Define functions to categorize frequency into ratings for online and offline data

def rate_frequency_online(frequency):
    if frequency > 3:
        return 5
    elif frequency == 3:
        return 4
    elif frequency == 2:
        return 3
    elif frequency > 1:
        return 2
    else:
        return 1

def rate_frequency_offline(frequency):
    if frequency > 20:
        return 5
    elif frequency <= 20 and frequency > 10:
        return 4
    elif frequency <= 10 and frequency > 4:
        return 3
    elif frequency <= 4 and frequency > 1:
        return 2
    else:
        return 1

# Apply the functions to the consolidated_data DataFrame
consolidated_data['Rating F_Online'] = consolidated_data['Frequency_Online'].apply(rate_frequency_online)
consolidated_data['Rating F_Offline'] = consolidated_data['Frequency_Offline'].apply(rate_frequency_offline)
consolidated_data

,email,GUID,orderTime_Online,orderTime_Offline,Months_Online,Months_Offline,Recency_Online,Recency_Offline,Frequency_Online,Frequency_Offline,Monetary_Online,Monetary_Offline,Rating R_Online,Rating R_Offline,Rating F_Online,Rating F_Offline
0,07zaf59@gmail.com,4325D934-6F38-4999-A87C-57605DC80B56,2023-03-01,2023-06-03,0-3,0-3,121,27,2,15,31.10,351.96,4,5,3,4
1,09webmail@gmail.com,B2206CC0-A8FC-4FFF-8FCB-D907DEC484D6,2023-06-16,2023-04-05,0-3,0-3,14,86,3,13,176.50,329.96,5,5,4,4
2,0roland.eberle0@gmail.com,7256FB03-2837-4D8D-9A9C-E2D65715CEC9,2022-11-15,2022-03-12,6-9,12-15,227,475,2,1,128.30,19.80,3,1,3,1
3,100571@gmx.ch,B0B3E49A-49B8-48A2-ACB2-1B7DBACA1697,2022-11-27,2022-11-04,6-9,6-9,215,238,2,8,138.20,180.12,4,3,3,3
4,1016@gmx.ch,64B02A11-8921-4546-84E5-8CDC917BD251,2022-07-03,2022-05-27,9-12,12-15,362,399,1,3,170.50,110.94,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28392,zwiker@gmx.ch,2F45C76A-B3D9-4DB3-8A29-B8CE8CC21B52,2023-04-12,2023-04-08,0-3,0-3,79,83,1,3,59.40,147.47,5,5,1,2
28393,zwimpfer.ruth@bluewin.ch,509E82ED-5366-44A2-B3AC-23ABE884DB28,2023-05-20,2023-03-25,0-3,0-3,41,97,19,8,1942.57,384.00,5,5,5,3
28394,zwyer-zettel@bluewin.ch,DAC6274F-A7DD-4BC7-BE79-7AC9AC05BC05,2022-05-05,2023-04-08,12-15,0-3,421,83,1,2,75.20,69.30,2,5,1,2
28395,zy.ame@hotmail.ch,EBFE76CE-E94C-423C-B257-CB7BE3D18472,2022-06-09,2022-11-15,9-12,6-9,386,227,1,2,139.00,17.80,2,3,1,2


In [ ]:
# Check the distribution for Monetary_Online
monetary_online_distribution = consolidated_data['Monetary_Online'].describe()

# Check the distribution for Monetary_Offline
monetary_offline_distribution = consolidated_data['Monetary_Offline'].describe()

monetary_online_distribution, monetary_offline_distribution


(count    28397.000000
 mean       221.445861
 std        285.995561
 min          0.000000
 25%         49.500000
 50%        134.600000
 75%        279.450000
 max      11960.800000
 Name: Monetary_Online, dtype: float64,
 count    28397.000000
 mean       383.389088
 std        401.641072
 min          0.000000
 25%        102.670000
 50%        272.010000
 75%        541.840000
 max      16988.860000
 Name: Monetary_Offline, dtype: float64)

In [ ]:
# Define function to categorize online monetary
def categorize_monetary_online(value):
    if value >= 279.45:
        return 5
    elif value >= 134.60:
        return 4
    elif value >= 70:
        return 3
    elif value >= 20:
        return 2
    else:
        return 1

# Define function to categorize offline monetary
def categorize_monetary_offline(value):
    if value >= 541.84:
        return 5
    elif value >= 272.01:
        return 4
    elif value >= 150:
        return 3
    elif value >= 40:
        return 2
    else:
        return 1

# Apply the functions to the Monetary columns to create the Rating columns
consolidated_data['Rating M_Online'] = consolidated_data['Monetary_Online'].apply(categorize_monetary_online)
consolidated_data['Rating M_Offline'] = consolidated_data['Monetary_Offline'].apply(categorize_monetary_offline)
consolidated_data

,email,GUID,orderTime_Online,orderTime_Offline,Months_Online,Months_Offline,Recency_Online,Recency_Offline,Frequency_Online,Frequency_Offline,Monetary_Online,Monetary_Offline,Rating R_Online,Rating R_Offline,Rating F_Online,Rating F_Offline,Rating M_Online,Rating M_Offline
0,07zaf59@gmail.com,4325D934-6F38-4999-A87C-57605DC80B56,2023-03-01,2023-06-03,0-3,0-3,121,27,2,15,31.10,351.96,4,5,3,4,2,4
1,09webmail@gmail.com,B2206CC0-A8FC-4FFF-8FCB-D907DEC484D6,2023-06-16,2023-04-05,0-3,0-3,14,86,3,13,176.50,329.96,5,5,4,4,4,4
2,0roland.eberle0@gmail.com,7256FB03-2837-4D8D-9A9C-E2D65715CEC9,2022-11-15,2022-03-12,6-9,12-15,227,475,2,1,128.30,19.80,3,1,3,1,3,1
3,100571@gmx.ch,B0B3E49A-49B8-48A2-ACB2-1B7DBACA1697,2022-11-27,2022-11-04,6-9,6-9,215,238,2,8,138.20,180.12,4,3,3,3,4,3
4,1016@gmx.ch,64B02A11-8921-4546-84E5-8CDC917BD251,2022-07-03,2022-05-27,9-12,12-15,362,399,1,3,170.50,110.94,2,2,1,2,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28392,zwiker@gmx.ch,2F45C76A-B3D9-4DB3-8A29-B8CE8CC21B52,2023-04-12,2023-04-08,0-3,0-3,79,83,1,3,59.40,147.47,5,5,1,2,2,2
28393,zwimpfer.ruth@bluewin.ch,509E82ED-5366-44A2-B3AC-23ABE884DB28,2023-05-20,2023-03-25,0-3,0-3,41,97,19,8,1942.57,384.00,5,5,5,3,5,4
28394,zwyer-zettel@bluewin.ch,DAC6274F-A7DD-4BC7-BE79-7AC9AC05BC05,2022-05-05,2023-04-08,12-15,0-3,421,83,1,2,75.20,69.30,2,5,1,2,3,2
28395,zy.ame@hotmail.ch,EBFE76CE-E94C-423C-B257-CB7BE3D18472,2022-06-09,2022-11-15,9-12,6-9,386,227,1,2,139.00,17.80,2,3,1,2,4,1


In [ ]:
# Calculate rank for Monetary_Online and Monetary_Offline
consolidated_data['Rank M_Online'] = consolidated_data['Monetary_Online'].rank(method='min', ascending=False).astype(int)
consolidated_data['Rank M_Offline'] = consolidated_data['Monetary_Offline'].rank(method='min', ascending=False).astype(int)
consolidated_data

,email,GUID,orderTime_Online,orderTime_Offline,Months_Online,Months_Offline,Recency_Online,Recency_Offline,Frequency_Online,Frequency_Offline,Monetary_Online,Monetary_Offline,Rating R_Online,Rating R_Offline,Rating F_Online,Rating F_Offline,Rating M_Online,Rating M_Offline,Rank M_Online,Rank M_Offline
0,07zaf59@gmail.com,4325D934-6F38-4999-A87C-57605DC80B56,2023-03-01,2023-06-03,0-3,0-3,121,27,2,15,31.10,351.96,4,5,3,4,2,4,23237,11608
1,09webmail@gmail.com,B2206CC0-A8FC-4FFF-8FCB-D907DEC484D6,2023-06-16,2023-04-05,0-3,0-3,14,86,3,13,176.50,329.96,5,5,4,4,4,4,12131,12272
2,0roland.eberle0@gmail.com,7256FB03-2837-4D8D-9A9C-E2D65715CEC9,2022-11-15,2022-03-12,6-9,12-15,227,475,2,1,128.30,19.80,3,1,3,1,3,1,14498,26681
3,100571@gmx.ch,B0B3E49A-49B8-48A2-ACB2-1B7DBACA1697,2022-11-27,2022-11-04,6-9,6-9,215,238,2,8,138.20,180.12,4,3,3,3,4,3,14044,17647
4,1016@gmx.ch,64B02A11-8921-4546-84E5-8CDC917BD251,2022-07-03,2022-05-27,9-12,12-15,362,399,1,3,170.50,110.94,2,2,1,2,4,2,12326,20858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28392,zwiker@gmx.ch,2F45C76A-B3D9-4DB3-8A29-B8CE8CC21B52,2023-04-12,2023-04-08,0-3,0-3,79,83,1,3,59.40,147.47,5,5,1,2,2,2,20483,19111
28393,zwimpfer.ruth@bluewin.ch,509E82ED-5366-44A2-B3AC-23ABE884DB28,2023-05-20,2023-03-25,0-3,0-3,41,97,19,8,1942.57,384.00,5,5,5,3,5,4,67,10700
28394,zwyer-zettel@bluewin.ch,DAC6274F-A7DD-4BC7-BE79-7AC9AC05BC05,2022-05-05,2023-04-08,12-15,0-3,421,83,1,2,75.20,69.30,2,5,1,2,3,2,18808,23166
28395,zy.ame@hotmail.ch,EBFE76CE-E94C-423C-B257-CB7BE3D18472,2022-06-09,2022-11-15,9-12,6-9,386,227,1,2,139.00,17.80,2,3,1,2,4,1,13975,27194
